In [1]:
# -*- coding: utf-8 -*-

#@author: alison

import re
import string
import pickle
import keras
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Model
from keras.layers import Dense, Dropout, Embedding, CuDNNGRU
from keras.layers import Input, GlobalAveragePooling1D
from keras.layers import Activation, GRU
from keras.optimizers import Adam, SGD, RMSprop
from keras import optimizers
from keras import regularizers

Using TensorFlow backend.


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
file_list = drive.ListFile({'q': "'1JsHac7_mxx_M8BapYSzcDKnzl4M9e1Yb' in parents and trashed=false"}).GetList()
for file1 in file_list:
    print('title: %s, id: %s' % (file1['title'], file1['id']))

title: gru_en_b.ipynb, id: 1l3cd4flv4i1iOwKjLdnDb5082nNkqvYe
title: bi_lstm_en_b.ipynb, id: 19N0cZ6WKYpnTfDVgKn1i-4UzLEN4jmSK
title: lstm_en_b.ipynb, id: 1n63dDEnPeqMTASzhwnaDnG_8knVqmIkX
title: WordEmbedding_GRU_en_b.ipynb, id: 1f87RaAgRU_RYsrSXZoHMu7ylpMv__0IV
title: CNN_en_b.ipynb, id: 1Y4-iz6jsFtExSbQu4Iw0Rdwz23cAcZA2
title: WordEmbedding_biLSTM_en_b.ipynb, id: 1Fhw6KwvnKCgzkAuZtDGhCXUs3CGfAYNS
title: WordEmbedding_LSTM_en_b.ipynb, id: 1F3PAN94OKvfELzEPdCrqFsUVLDlwpcib
title: test_en.tsv, id: 1Z-kJ95PF2VZiGAn_5Piyovk8SZ-bYQJv
title: test_es.tsv, id: 1PlU_aDjY9RLq2CKW1zMQTDmRDcZ0f8YL
title: train_en.tsv, id: 1tv4kRZtCJu7F4WtkBvaMWZMXZZDYF-yv
title: dev_en.tsv, id: 116R1Q_P_m75ZagpkE7cZXfXHiDSDwrJy
title: train_es.tsv, id: 18SENOaqh8YZ98vrzHgHP3sQk3sycMz0H
title: dev_es.tsv, id: 1S5cZDZq7Mhmxpp_SUs79j86An3ICs265


In [0]:
train_downloaded = drive.CreateFile({'id': '1tv4kRZtCJu7F4WtkBvaMWZMXZZDYF-yv'})
train_downloaded.GetContentFile('train_en.tsv')
dev_downloaded = drive.CreateFile({'id': '116R1Q_P_m75ZagpkE7cZXfXHiDSDwrJy'})
dev_downloaded.GetContentFile('dev_en.tsv')
test_downloaded = drive.CreateFile({'id': '1Z-kJ95PF2VZiGAn_5Piyovk8SZ-bYQJv'})
test_downloaded.GetContentFile('test_en.tsv')

In [0]:
train = pd.read_csv('train_en.tsv', delimiter='\t',encoding='utf-8')
dev = pd.read_csv('dev_en.tsv', delimiter='\t',encoding='utf-8')
#test = pd.read_csv('test.tsv', delimiter='\t',encoding='utf-8')

In [0]:
# Etapa de pré-processamento

def clean_tweets(tweet):
    tweet = re.sub('@(\\w{1,15})\b', '', tweet)
    tweet = tweet.replace("via ", "")
    tweet = tweet.replace("RT ", "")
    tweet = tweet.lower()
    return tweet
    
def clean_url(tweet):
    tweet = re.sub('http\\S+', '', tweet, flags=re.MULTILINE)   
    return tweet
    
def remove_stop_words(tweet):
    stops = set(stopwords.words("english"))
    stops.update(['.',',','"',"'",'?',':',';','(',')','[',']','{','}'])
    toks = [tok for tok in tweet if not tok in stops and len(tok) >= 3]
    return toks
    
def stemming_tweets(tweet):
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in tweet]
    return stemmed_words

def remove_number(tweet):
    newTweet = re.sub('\\d+', '', tweet)
    return newTweet

def remove_hashtags(tweet):
    result = ''

    for word in tweet.split():
        if word.startswith('#') or word.startswith('@'):
            result += word[1:]
            result += ' '
        else:
            result += word
            result += ' '

    return result

In [0]:
def preprocessing(tweet, swords = True, url = True, stemming = True, ctweets = True, number = True, hashtag = True):

    if ctweets:
        tweet = clean_tweets(tweet)

    if url:
        tweet = clean_url(tweet)

    if hashtag:
        tweet = remove_hashtags(tweet)
    
    twtk = TweetTokenizer(strip_handles=True, reduce_len=True)

    if number:
        tweet = remove_number(tweet)
    
    tokens = [w.lower() for w in twtk.tokenize(tweet) if w != "" and w is not None]

    if swords:
        tokens = remove_stop_words(tokens)

    if stemming:
        tokens = stemming_tweets(tokens)

    text = " ".join(tokens)

    return text

In [0]:
train_text  = train['text'].map(lambda x: preprocessing(x, swords = True, url = True, stemming = True, ctweets = True, number = True, hashtag = True))
hs_train    = train['HS']
id_train    = train['id']
tr_train    = train['TR']
ag_train    = [1 if k == '1' else 0 for k in train['AG']]

test_text  = dev['text'].map(lambda x: preprocessing(x, swords = True, url = True, stemming = True, ctweets = True, number = True, hashtag = True))
hs_test    = dev['HS']
id_test    = dev['id']
tr_test    = dev['TR']
ag_test    = dev['AG']

max_features = 25000
maxlen = 100

In [0]:
def code_data(train_text, y_train, test_text, y_test, maxlen, max_features):
    data = np.concatenate((train_text, test_text), axis=0)
    classes = np.concatenate((y_train, y_test), axis=0)

    # Treina um tokenizaddor nos dados de treino
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(data)

    # Tokeniza os dados
    X = tokenizer.texts_to_sequences(data)
    Y = tokenizer.texts_to_sequences(test_text)

    tweets = sequence.pad_sequences(X, maxlen=maxlen)
    x_test  = sequence.pad_sequences(Y, maxlen=maxlen)
    
    return tweets, x_test, classes

In [0]:
# Fase de classificação de sentimentos
batch_size = 32
epochs = 5

tweet_input = Input(shape=(maxlen,), dtype='int32')

embedding = Embedding(max_features, 256, input_length=maxlen, trainable=True)(tweet_input)

gru1 = CuDNNGRU(256, kernel_initializer='normal', return_sequences=True)(embedding)

gru2 = CuDNNGRU(256)(gru1)

dens = Dense(256, activation='relu')(gru2)

output = Dense(1, activation='sigmoid')(dens)

model = Model(inputs=tweet_input, outputs=output)

#opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.9)
#opt = SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
opt = RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.01)

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [60]:
# Treinamento e predição - Hate Speech (HS)

tweets, x_test, classes = code_data(train_text, hs_train, test_text, hs_test, maxlen, max_features)

x_train, x_val, y_train, y_val = train_test_split(tweets, classes, test_size=0.25, random_state=None)

model.summary()

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True, validation_data=(x_val, y_val))

hs = (model.predict(x_test, batch_size=batch_size) > .5).astype(int)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 100, 256)          6400000   
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (None, 100, 256)          394752    
_________________________________________________________________
cu_dnngru_2 (CuDNNGRU)       (None, 256)               394752    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 7,255,553
Trainable params: 7,255,553
Non-trainable params: 0
_________________________________________________________________


In [61]:
# Treinamento e predição - Target Range (TR)

tweets, x_test, classes = code_data(train_text, tr_train, test_text, tr_test, maxlen, max_features)

x_train, x_val, y_train, y_val = train_test_split(tweets, classes, test_size=0.25, random_state=None)

model.summary()

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True, validation_data=(x_val, y_val))

tr = (model.predict(x_test, batch_size=batch_size) > .5).astype(int)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 100, 256)          6400000   
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (None, 100, 256)          394752    
_________________________________________________________________
cu_dnngru_2 (CuDNNGRU)       (None, 256)               394752    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 7,255,553
Trainable params: 7,255,553
Non-trainable params: 0
_________________________________________________________________


In [62]:
# Treinamento e predição - Aggressiveness (AG)

tweets, x_test, classes = code_data(train_text, ag_train, test_text, ag_test, maxlen, max_features)

x_train, x_val, y_train, y_val = train_test_split(tweets, classes, test_size=0.25, random_state=None)

model.summary()

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True, validation_data=(x_val, y_val))

ag = (model.predict(x_test, batch_size=batch_size) > .5).astype(int)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 100, 256)          6400000   
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (None, 100, 256)          394752    
_________________________________________________________________
cu_dnngru_2 (CuDNNGRU)       (None, 256)               394752    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 7,255,553
Trainable params: 7,255,553
Non-trainable params: 0
_________________________________________________________________


In [63]:
def func(pred):
    output = []
    
    for array in pred:
        output.append(array[0])
    
    return output


hs_pred = func(hs)
tr_pred = func(tr)
ag_pred = func(ag)

print(len(hs_pred), len(tr_pred), len(ag_pred))

1000 1000 1000


In [0]:
from google.colab import files

with open("en_b.tsv", "w") as file:
    for i in range(len(hs_pred)):
        file.write(str(id_test[i]))
        file.write('\t')
        file.write(str(hs_pred[i]))
        file.write('\t')
        file.write(str(tr_pred[i]))
        file.write('\t')
        file.write(str(ag_pred[i]))
        file.write('\n')
    
files.download('en_b.tsv')